<a href="https://colab.research.google.com/github/dagyeom23658/deeplearning_master/blob/main/seq2seq_%2C_%EC%96%B4%ED%85%90%EC%85%98_%EB%A9%94%EC%BB%A4%EB%8B%88%EC%A6%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNN을 이용한 인코더-디코더

-  케라스 함수형 API(https://wikidocs.net/38861)

## 시퀀스-투-시퀀스(Sequence-to-Sequence)

시퀀스-투-시퀀스(Sequence-to-Sequence)는 입력된 시퀀스로부터 다른 도메인의 시퀀스를 출력하는 다양한 분야에서 사용되는 모델입니다. 예를 들어 챗봇(Chatbot)과 기계 번역(Machine Translation)이 그러한 대표적인 예인데, 입력 시퀀스와 출력 시퀀스를 각각 질문과 대답으로 구성하면 챗봇으로 만들 수 있고, 입력 시퀀스와 출력 시퀀스를 각각 입력 문장과 번역 문장으로 만들면 번역기로 만들 수 있습니다. 그 외에도 내용 요약(Text Summarization), STT(Speech to Text) 등에서 쓰일 수 있습니다.

- 인코더는 입력 문장의 모든 단어들을 순차적으로 입력받은 뒤에 마지막에 이 모든 단어 정보들을 압축해서 하나의 벡터로 만드는데, 이를 컨텍스트 벡터(context vector)라고 합니다. 입력 문장의 정보가 하나의 컨텍스트 벡터로 모두 압축되면 인코더는 컨텍스트 벡터를 디코더로 전송합니다. 디코더는 컨텍스트 벡터를 받아서 번역된 단어를 한 개씩 순차적으로 출력합니다.

- 인코더 아키텍처와 디코더 아키텍처의 내부는 사실 두 개의 RNN 아키텍처 입니다. 입력 문장을 받는 RNN 셀을 인코더라고 하고, 출력 문장을 출력하는 RNN 셀을 디코더라고 합니다.

- 인코더 RNN 셀은 모든 단어를 입력받은 뒤에 인코더 RNN 셀의 마지막 시점의 은닉 상태를 디코더 RNN 셀로 넘겨주는데 이를 컨텍스트 벡터라고 합니다. 컨텍스트 벡터는 디코더 RNN 셀의 첫번째 은닉 상태로 사용됩니다.

- 디코더는 초기 입력으로 문장의 시작을 의미하는 심볼 <sos>가 들어갑니다. 디코더는 <sos>가 입력되면, 다음에 등장할 확률이 높은 단어를 예측합니다. 첫번째 시점(time step)의 디코더 RNN 셀은 다음에 등장할 단어로 je를 예측하였습니다. 첫번째 시점의 디코더 RNN 셀은 예측된 단어 je를 다음 시점의 RNN 셀의 입력으로 입력합니다. 그리고 두번째 시점의 디코더 RNN 셀은 입력된 단어 je로부터 다시 다음에 올 단어인 suis를 예측하고, 또 다시 이것을 다음 시점의 RNN 셀의 입력으로 보냅니다. 디코더는 이런 식으로 기본적으로 다음에 올 단어를 예측하고, 그 예측한 단어를 다음 시점의 RNN 셀의 입력으로 넣는 행위를 반복합니다. 이 행위는 문장의 끝을 의미하는 심볼인 <eos>가 다음 단어로 예측될 때까지 반복됩니다.

- 출력 단어로 나올 수 있는 단어들은 다양한 단어들이 있습니다. seq2seq 모델은 선택될 수 있는 모든 단어들로부터 하나의 단어를 골라서 예측해야 합니다.  디코더에서 각 시점(time step)의 RNN 셀에서 출력 벡터가 나오면, 해당 벡터는 소프트맥스 함수를 통해 출력 시퀀스의 각 단어별 확률값을 반환하고, 디코더는 출력 단어를 결정합니다.

### 글자 레벨 기계 번역기(Character-Level Neural Machine Translation) 구현하기
sequence-to-sequence 10분만에 이해하기 : https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html

기계 번역기를 훈련시키기 위해서는 훈련 데이터로 병렬 코퍼스(parallel corpus)가 필요합니다. 병렬 코퍼스란, 두 개 이상의 언어가 병렬적으로 구성된 코퍼스를 의미합니다.

https://lsjsj92.tistory.com/493

In [49]:
import os
import shutil
import zipfile

import pandas as pd
import tensorflow as tf
import urllib3
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [50]:
http = urllib3.PoolManager()
url ='http://www.manythings.org/anki/kor-eng.zip'
filename = 'kor-eng.zip'
path = os.getcwd()
zipfilename = os.path.join(path, filename)
with http.request('GET', url, preload_content=False) as r, open(zipfilename, 'wb') as out_file:       
    shutil.copyfileobj(r, out_file)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
    zip_ref.extractall(path)

In [51]:
lines = pd.read_csv('kor.txt', names=['src', 'tar', 'lic'], sep='\t')  #src는 source의 줄임말로 입력 문장을 나타내며, tar는 target의 줄임말로 번역하고자 하는 문장
print(lines)
del lines['lic']
print('전체 샘플의 개수 :',len(lines))

                                                    src  ...                                                lic
0                                                   Go.  ...  CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1                                                   Hi.  ...  CC-BY 2.0 (France) Attribution: tatoeba.org #5...
2                                                  Run!  ...  CC-BY 2.0 (France) Attribution: tatoeba.org #9...
3                                                  Run.  ...  CC-BY 2.0 (France) Attribution: tatoeba.org #4...
4                                                  Who?  ...  CC-BY 2.0 (France) Attribution: tatoeba.org #2...
...                                                 ...  ...                                                ...
3725  Science fiction has undoubtedly been the inspi...  ...  CC-BY 2.0 (France) Attribution: tatoeba.org #6...
3726  I started a new blog. I'll do my best not to b...  ...  CC-BY 2.0 (France) Attribution: tatoeba.or

In [52]:
lines = lines.loc[:, 'src':'tar']
# lines = lines[0:60000] # 6만개만 저장
lines.sample(10)

,src,tar
555,We can buy it.,이건 우리가 살 수 있어.
2683,I don't like sitting on the floor.,난 바닥에 앉는 건 별로야.
2314,Tom wants to avoid everything.,톰은 모든 걸 회피하고 싶어해.
72,Go ahead.,계속해.
1040,Please drive slowly.,천천히 운전하세요.
350,Tom cheated.,톰이 사기 쳤어.
214,Fire burns.,불타네.
1447,I want to see you smile.,난 당신이 웃는 걸 보고 싶어요.
2260,I want to get there by subway.,지하철로 가고 싶어.
2948,I think Mary is way cuter than Alice.,메리가 앨리스보다 좀 더 귀여운 것 같아.


In [53]:
# 번역 문장에 해당되는 한국어 데이터는 앞서 배웠듯이 시작을 의미하는 심볼 <sos>과 종료를 의미하는 심볼 <eos>을 넣어주어야 합니다. 
# 여기서는 <sos>와 <eos> 대신 '\t'를 시작 심볼, '\n'을 종료 심볼로 간주하여 추가하고 다시 데이터를 출력해보겠습니다.

lines.tar = lines.tar.apply(lambda x : '\t '+ x + ' \n')
lines.sample(10)

,src,tar
1380,Tom is a great manager.,\t 톰은 관리를 정말 잘해. \n
860,Life is too short.,\t 삶은 너무 짧네. \n
2672,Every country has its own history.,\t 어떤 나라든 역사가 있다. \n
1170,This room is too big.,\t 이 방은 너무 커요. \n
2815,It's not as difficult as you think.,\t 그건 생각보단 어렵지 않아. \n
1980,I live in a small apartment.,\t 나는 작은 아파트에 살아요. \n
1549,Have you ever been drunk?,\t 취해본 적 있어? \n
2149,I've heard this joke already.,\t 이 농담은 이미 들어봤어. \n
2293,Tom and Mary are both lawyers.,\t 톰과 메리는 둘 다 변호사야. \n
2470,I know I'm going to learn a lot.,\t 내가 많이 배우게 될 거라는 걸 알고 있어. \n


In [54]:
# 글자 집합 구축(토큰 단위가 단어가 아니라 글자)
src_vocab = set()
for line in lines.src: # 1줄씩 읽음
    for char in line: # 1개의 글자씩 읽음
        src_vocab.add(char)

tar_vocab = set()   # set : https://wikidocs.net/16044
for line in lines.tar: 
    for char in line:
        tar_vocab.add(char)

In [55]:
src_vocab

{' ',
 '!',
 '"',
 '$',
 '%',
 "'",
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '°',
 'ï'}

In [56]:
tar_vocab

{'\t',
 '\n',
 ' ',
 '!',
 '"',
 '%',
 '(',
 ')',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'H',
 'M',
 'N',
 'T',
 'a',
 'd',
 'h',
 'i',
 'm',
 'o',
 'p',
 'r',
 't',
 'y',
 '°',
 '가',
 '각',
 '간',
 '갇',
 '갈',
 '감',
 '갑',
 '값',
 '갔',
 '강',
 '갖',
 '같',
 '개',
 '객',
 '갰',
 '걀',
 '걔',
 '거',
 '걱',
 '건',
 '걷',
 '걸',
 '검',
 '겁',
 '것',
 '게',
 '겐',
 '겠',
 '겨',
 '격',
 '겪',
 '견',
 '결',
 '겼',
 '경',
 '계',
 '고',
 '곡',
 '곤',
 '곧',
 '골',
 '곰',
 '곱',
 '곳',
 '공',
 '과',
 '관',
 '광',
 '괜',
 '괴',
 '굉',
 '교',
 '구',
 '국',
 '군',
 '굳',
 '굴',
 '굶',
 '굼',
 '굽',
 '궁',
 '권',
 '귀',
 '귄',
 '규',
 '그',
 '극',
 '근',
 '글',
 '금',
 '급',
 '긋',
 '긍',
 '기',
 '긴',
 '길',
 '깊',
 '까',
 '깎',
 '깐',
 '깔',
 '깜',
 '깡',
 '깨',
 '꺼',
 '꺾',
 '껍',
 '껏',
 '껐',
 '께',
 '껴',
 '꼈',
 '꼬',
 '꼴',
 '꼼',
 '꽃',
 '꽉',
 '꽤',
 '꾸',
 '꾼',
 '꿇',
 '꿈',
 '꿔',
 '꿨',
 '뀌',
 '끄',
 '끈',
 '끊',
 '끌',
 '끓',
 '끔',
 '끗',
 '끙',
 '끝',
 '끼',
 '낀',
 '낄',
 '낌',
 '나',
 '낙',
 '낚',
 '난',
 '날',
 '

In [57]:
# 글자 집합의 크기
src_vocab_size = len(src_vocab)+1
tar_vocab_size = len(tar_vocab)+1
print('source 문장의 char 집합 :',src_vocab_size)
print('target 문장의 char 집합 :',tar_vocab_size)

source 문장의 char 집합 : 75
target 문장의 char 집합 : 914


In [58]:
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))
print(src_vocab[45:75])
print(tar_vocab[45:75])

['Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '°', 'ï']
['간', '갇', '갈', '감', '갑', '값', '갔', '강', '갖', '같', '개', '객', '갰', '걀', '걔', '거', '걱', '건', '걷', '걸', '검', '겁', '것', '게', '겐', '겠', '겨', '격', '겪', '견']


In [59]:
# 각 글자에 인덱스를 부여

src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
print(src_to_index)
print(tar_to_index)

{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, "'": 6, ',': 7, '-': 8, '.': 9, '0': 10, '1': 11, '2': 12, '3': 13, '4': 14, '5': 15, '6': 16, '7': 17, '8': 18, '9': 19, ':': 20, ';': 21, '?': 22, 'A': 23, 'B': 24, 'C': 25, 'D': 26, 'E': 27, 'F': 28, 'G': 29, 'H': 30, 'I': 31, 'J': 32, 'K': 33, 'L': 34, 'M': 35, 'N': 36, 'O': 37, 'P': 38, 'Q': 39, 'R': 40, 'S': 41, 'T': 42, 'U': 43, 'V': 44, 'W': 45, 'Y': 46, 'a': 47, 'b': 48, 'c': 49, 'd': 50, 'e': 51, 'f': 52, 'g': 53, 'h': 54, 'i': 55, 'j': 56, 'k': 57, 'l': 58, 'm': 59, 'n': 60, 'o': 61, 'p': 62, 'q': 63, 'r': 64, 's': 65, 't': 66, 'u': 67, 'v': 68, 'w': 69, 'x': 70, 'y': 71, 'z': 72, '°': 73, 'ï': 74}
{'\t': 1, '\n': 2, ' ': 3, '!': 4, '"': 5, '%': 6, '(': 7, ')': 8, ',': 9, '-': 10, '.': 11, '/': 12, '0': 13, '1': 14, '2': 15, '3': 16, '4': 17, '5': 18, '6': 19, '7': 20, '8': 21, '9': 22, ':': 23, '?': 24, 'A': 25, 'B': 26, 'C': 27, 'D': 28, 'H': 29, 'M': 30, 'N': 31, 'T': 32, 'a': 33, 'd': 34, 'h': 35, 'i': 36, 'm': 37, 'o': 38, 'p': 3

In [60]:
# 1) 영어 문장에 대한 정수 인코딩을 수행

encoder_input = []

# 1개의 문장
for line in lines.src:  # 한개의 문장을 뽑아낸다. 
  encoded_line = []
  # 각 줄에서 1개의 char
  for char in line:  # 한 개의 문장에 대해 정수인코딩.
    # 각 char을 정수로 변환
    encoded_line.append(src_to_index[char])
  encoder_input.append(encoded_line)   # 정수인코딩된 문장을 인코더input리스트에 넣는다. 
print('source 문장의 정수 인코딩 :',encoder_input[:5])

source 문장의 정수 인코딩 : [[29, 61, 9], [30, 55, 9], [40, 67, 60, 2], [40, 67, 60, 9], [45, 54, 61, 22]]


In [61]:
# 2) 한국어 타겟데이터에 대해서 정수 인코딩
decoder_input = []
for line in lines.tar:
  encoded_line = []
  for char in line:
    encoded_line.append(tar_to_index[char])
  decoder_input.append(encoded_line)
print('target 문장의 정수 인코딩 :',decoder_input[:5])

# 시작 심볼, 종료심볼인 1과 2로 감싸져 있음을 볼수 있음.

target 문장의 정수 인코딩 : [[1, 3, 44, 11, 3, 2], [1, 3, 550, 195, 11, 3, 2], [1, 3, 288, 572, 4, 3, 2], [1, 3, 288, 572, 11, 3, 2], [1, 3, 206, 96, 24, 3, 2]]


In [62]:
#  3) 디코더의 예측값과 비교하기 위한 실제값 정수인코딩(시작심볼이 필요없음.-> 문장의 맨 앞에 붙어있는 '\t'를 제거)
decoder_target = []
for line in lines.tar:
  timestep = 0
  encoded_line = []
  for char in line:
    if timestep > 0:
      encoded_line.append(tar_to_index[char])
    timestep = timestep + 1
  decoder_target.append(encoded_line)
print('target 문장 레이블의 정수 인코딩 :',decoder_target[:5])

target 문장 레이블의 정수 인코딩 : [[3, 44, 11, 3, 2], [3, 550, 195, 11, 3, 2], [3, 288, 572, 4, 3, 2], [3, 288, 572, 11, 3, 2], [3, 206, 96, 24, 3, 2]]


모델을 설계하기 전에 혹시 의아한 점은 없으신가요? 현재 시점의 디코더 셀의 입력은 오직 이전 디코더 셀의 출력을 입력으로 받는다고 설명하였는데 decoder_input이 왜 필요할까요?

훈련 과정에서는 이전 시점의 디코더 셀의 출력을 현재 시점의 디코더 셀의 입력으로 넣어주지 않고, 이전 시점의 실제값을 현재 시점의 디코더 셀의 입력값으로 하는 방법을 사용할 겁니다. 그 이유는 이전 시점의 디코더 셀의 예측이 틀렸는데 이를 현재 시점의 디코더 셀의 입력으로 사용하면 현재 시점의 디코더 셀의 예측도 잘못될 가능성이 높고 이는 연쇄 작용으로 디코더 전체의 예측을 어렵게 합니다. 이런 상황이 반복되면 훈련 시간이 느려집니다. 만약 이 상황을 원하지 않는다면 이전 시점의 디코더 셀의 예측값 대신 실제값을 현재 시점의 디코더 셀의 입력으로 사용하는 방법을 사용할 수 있습니다. 이와 같이 RNN의 모든 시점에 대해서 이전 시점의 예측값 대신 실제값을 입력으로 주는 방법을 교사 강요라고 합니다.

In [64]:
#  패딩을 위해서 영어 문장과 프랑스어 문장 각각에 대해서 가장 길이가 긴 샘플의 길이를 확인합니다.
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])
print('source 문장의 최대 길이 :',max_src_len)
print('target 문장의 최대 길이 :',max_tar_len)

# 영어와 프랑스어의 길이는 하나의 쌍이라고 하더라도 전부 다르므로 
# 패딩을 할 때도 이 두 개의 데이터의 길이를 전부 동일하게 맞춰줄 필요는 없습니다. 영어 데이터는 영어 샘플들끼리, 프랑스어는 프랑스어 샘플들끼리 길이를 맞추어서 패딩하면 됩니다. 
# 여기서는 가장 긴 샘플의 길이에 맞춰서 영어 데이터의 샘플은 전부 길이가 23이 되도록 패딩하고, 프랑스어 데이터의 샘플은 전부 길이가 76이 되도록 패딩합니다.

source 문장의 최대 길이 : 537
target 문장의 최대 길이 : 300


In [39]:
for line in lines.tar:
    if len(line) == max_tar_len:
        print(line)

	 의심의 여지 없이 세상에는 어떤 남자이든 정확히 딱 알맞는 여자와 결혼하거나 그 반대의 상황이 존재하지. 그런데 인간이 수백 명의 사람만 알고 지내는 사이가 될 기회를 갖는다고 생각해 보면, 또 그 수백 명 중 열여 명 쯤 이하만 잘 알 수 있고, 그리고 나서 그 열여 명 중에 한두 명만 친구가 될 수 있다면, 그리고 또 만일 우리가 이 세상에 살고 있는 수백만 명의 사람들만 기억하고 있다면, 딱 맞는 남자는 지구가 생겨난 이래로 딱 맞는 여자를 단 한번도 만난 적이 없을 수도 있을 거라는 사실을 쉽게 눈치챌 수 있을 거야. 



In [40]:
for line in lines.src:
    if len(line) == max_src_len:
        print(line)

# 문제의 원인!
# 아마 하나의 고정된 크기의 벡터에 모든 정보를 압축하려고 하니까 정보 손실이 발생떄문에 번역이 제대로 안된 것 같음.
# 그것도 그렇고... 영어와 불어라면 모를까, 한국어와 영어는 어휘체계가 달라서 글자 단위가 아닌 단어단위로 해야할 것 같음. --> 기계 번역 분야에서 입력 문장이 길면 번역 품질이 떨어지는 현상 --> 어텐션 등장.
# https://wikidocs.net/86900

Doubtless there exists in this world precisely the right woman for any given man to marry and vice versa; but when you consider that a human being has the opportunity of being acquainted with only a few hundred people, and out of the few hundred that there are but a dozen or less whom he knows intimately, and out of the dozen, one or two friends at most, it will easily be seen, when we remember the number of millions who inhabit this world, that probably, since the earth was created, the right man has never yet met the right woman.


In [24]:
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

In [25]:
# 글자 단위 번역기므로 워드 임베딩은 별도로 사용되지 않으며, 예측값과의 오차 측정에 사용되는 실제값뿐만 아니라 입력값도 원-핫 벡터를 사용하겠습니다.
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

In [26]:
# seq2seq 모델을 설계 - 교사 강요를 사용하여 훈련

from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
import numpy as np

In [28]:
#  functional API를 사용
encoder_inputs = Input(shape=(None, src_vocab_size))  #src_vocab_size : 영어글자 집합의 크기
encoder_lstm = LSTM(units=256, return_state=True)  # return_state=True 마지막 시점의 은닉상태출력. 인코더의 내부 상태를 디코더로 넘겨주어야 하기 때문에 return_state=True로 설정

# encoder_outputs은 여기서는 불필요,은닉상태, 셀상태만 전달하면 된다. 
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

# LSTM은 바닐라 RNN과는 달리 상태가 두 개. 은닉 상태와 셀 상태. LSTM은 은닉 상태와 셀 상태라는 두 가지 상태를 가진다
encoder_states = [state_h, state_c]

#  encoder_states를 디코더에 전달하므로서 이 두 가지 상태 모두를 디코더로 전달합니다. 이것이 앞서 배운 컨텍스트 벡터입니다.

In [31]:
decoder_inputs = Input(shape=(None, tar_vocab_size))  #tar_vocab_size : 한국글자 집합크기
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True) #동일하게 디코더의 은닉 상태 크기도 256

# 디코더에게 인코더의 은닉 상태, 셀 상태를 전달.
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state=encoder_states)  # initial_state=encoder_states : 디코더는 인코더의 마지막 은닉 상태를 초기 은닉 상태로 사용합  디코더도 은닉 상태, 셀 상태를 리턴하기는 하지만 훈련 과정에서는 사용x
decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')  #출력층에 한국어의 단어 집합의 크기만큼 뉴런을 배치한 후 소프트맥스 함수를 사용하여 실제값과의 오차를 구합니다.
decoder_outputs = decoder_softmax_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

In [32]:
# 1. 번역하고자 하는 입력 문장이 인코더에 들어가서 은닉 상태와 셀 상태를 얻습니다.
# 2. 상태와 <SOS>에 해당하는 '\t'를 디코더로 보냅니다.
# 3. 디코더가 <EOS>에 해당하는 '\n'이 나올 때까지 다음 문자를 예측하는 행동을 반복합니다.

In [ ]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [33]:
#  encoder_inputs와 encoder_states는 훈련 과정에서 이미 정의한 것들을 재사용
# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용.
# 뒤의 함수 decode_sequence()에 동작을 구현 예정
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)

# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태를 버리지 않음.
decoder_states = [state_h, state_c]
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

In [36]:
# 단어로부터 인덱스를 얻는 것이 아니라 인덱스로부터 단어를 얻을 수 있는 index_to_src와 index_to_tar를 만들었습니다.
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

In [37]:
def decode_sequence(input_seq):
  # 입력으로부터 인코더의 상태를 얻음
  states_value = encoder_model.predict(input_seq)

  # <SOS>에 해당하는 원-핫 벡터 생성
  target_seq = np.zeros((1, 1, tar_vocab_size))
  target_seq[0, 0, tar_to_index['\t']] = 1.

  stop_condition = False
  decoded_sentence = ""

  # stop_condition이 True가 될 때까지 루프 반복
  while not stop_condition:
    # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

    # 예측 결과를 문자로 변환
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = index_to_tar[sampled_token_index]

    # 현재 시점의 예측 문자를 예측 문장에 추가
    decoded_sentence += sampled_char

    # <eos>에 도달하거나 최대 길이를 넘으면 중단.
    if (sampled_char == '\n' or
        len(decoded_sentence) > max_tar_len):
        stop_condition = True

    # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, sampled_token_index] = 1.

    # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
    states_value = [h, c]

  return decoded_sentence

In [38]:
for seq_index in [3,50,100,300,1001]: # 입력 문장의 인덱스
  input_seq = encoder_input[seq_index:seq_index+1]
  decoded_sentence = decode_sequence(input_seq)
  print(35 * "-")
  print('입력 문장:', lines.src[seq_index])
  print('정답 문장:', lines.tar[seq_index][2:len(lines.tar[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
  print('번역 문장:', decoded_sentence[1:len(decoded_sentence)-1]) # '\n'을 빼고 출력

-----------------------------------
입력 문장: Run.
정답 문장: 뛰어. 
번역 문장: 국,월월월C뉴뉴월월C뉴뉴탓탓탓육육려줬최가가와와와와와와와와셨셨늙셨늙왔챌챌브한챌챌브한마헉는던던비땋년밌깔홋돼aa떴몰뒷왼막읽샌섯뱀됐곧흙곧밑개마람당딱긴몇몇디봤취쪼넌구구씬났!취문짧건은낼모천웠롭롭꿔몰각각d윈과기맡료료료박엔료그그깊닥북북좀껏륙더운1떻냈루월월CC뉴뉴길월월월월C뉴뉴탓탓육육려르능똑찔곡축꿈옷괜삶국쿠섭어무작)매:썼만탁탁평묶묶뛰혼혼헌M늄압코낭젯감혼혼대대송송였였힘컵컵컵폐컵컵폐컵폐컵,밀플"난덕덕잔청청럽럽줄럽얻꽃특륜뭇젝꿈봤락헤렌내mm휴m휴m품품급읽줘줘줘앙낭mm품급읽랍천천만육자자명묶뛰묶젯묶젯혼헌혼헌등씀씬듯듯특후조황d거허의힌멈화곤곤곤곤텐썼덤현현섭뒤이
-----------------------------------
입력 문장: Help me.
정답 문장: 도와줘. 
번역 문장: 국,월월월C뉴뉴월월C뉴뉴탓탓탓육육려줬최가가와와와와와와와와셨셨늙셨늙왔챌챌브한챌챌브한마헉는던던비땋년밌깔홋돼aa떴몰뒷왼막읽샌섯뱀됐곧흙곧밑개마람당딱긴몇몇디봤취쪼넌구구씬났!취문짧건은낼모천웠롭롭꿔몰각각d윈과기맡료료료박엔료그그깊닥북북좀껏륙더운1떻냈루월월CC뉴뉴길월월월월C뉴뉴탓탓육육려르능똑찔곡축꿈옷괜삶국쿠섭어무작)매:썼만탁탁평묶묶뛰혼혼헌M늄압코낭젯감혼혼대대송송였였힘컵컵컵폐컵컵폐컵폐컵,밀플"난덕덕잔청청럽럽줄럽얻꽃특륜뭇젝꿈봤락헤렌내mm휴m휴m품품급읽줘줘줘앙낭mm품급읽랍천천만육자자명묶뛰묶젯묶젯혼헌혼헌등씀씬듯듯특후조황d거허의힌멈화곤곤곤곤텐썼덤현현섭뒤이
-----------------------------------
입력 문장: Tom lost.
정답 문장: 톰이 졌어. 
번역 문장: 국,월월월C뉴뉴월월C뉴뉴탓탓탓육육려줬최가가와와와와와와와와셨셨늙셨늙왔챌챌브한챌챌브한마헉는던던비땋년밌깔홋돼aa떴몰뒷왼막읽샌섯뱀됐곧흙곧밑개마람당딱긴몇몇디봤취쪼넌구구씬났!취문짧건은낼모천웠롭롭꿔몰각각d윈과기맡료료료박엔료그그깊닥북북좀껏륙더운1떻냈루월월CC뉴뉴길월월월월C뉴뉴탓탓육육려르능똑찔곡축꿈옷괜삶국쿠섭어무작)매:썼만탁탁평묶묶뛰혼혼헌M늄압코낭

KeyboardInterrupt: ignored

## 어텐션
- 어텐션의 기본 아이디어는 디코더에서 출력 단어를 예측하는 매 시점(time step)마다, 인코더에서의 전체 입력 문장을 다시 한 번 참고한다는 점입니다. 단, 전체 입력 문장을 전부 다 동일한 비율로 참고하는 것이 아니라, 해당 시점에서 예측해야할 단어와 연관이 있는 입력 단어 부분을 좀 더 집중(attention)해서 보게 됩니다.

- Attention(Q, K, V) = Attention Value

- 어텐션 함수는 주어진 '쿼리(Query)'에 대해서 모든 '키(Key)'와의 유사도를 각각 구합니다. 그리고 구해낸 이 유사도를 키와 맵핑되어있는 각각의 '값(Value)'에 반영해줍니다. 그리고 유사도가 반영된 '값(Value)'을 모두 더해서 리턴합니다. 여기서는 이를 어텐션 값(Attention Value)이라고 하겠습니다.

```
Q = Query : t 시점의 디코더 셀에서의 은닉 상태
K = Keys : 모든 시점의 인코더 셀의 은닉 상태들
V = Values : 모든 시점의 인코더 셀의 은닉 상태들
```

# 트랜스포머(Transformer)
https://wikidocs.net/31379

- "Attention is all you need"에서 나온 모델로 기존의 seq2seq의 구조인 인코더-디코더를 따르면서도, 논문의 이름처럼 어텐션(Attention)만으로 구현한 모델

이 모델은 RNN을 사용하지 않고, 인코더-디코더 구조를 설계하였음에도 성능도 RNN보다 우수하다는 특징을 갖고있습니다.


 ```
 dmodel = 512
트랜스포머의 인코더와 디코더에서의 정해진 입력과 출력의 크기를 의미합니다. 임베딩 벡터의 차원 또한 
이며, 각 인코더와 디코더가 다음 층의 인코더와 디코더로 값을 보낼 때에도 이 차원을 유지합니다. 논문에서는 512입니다.

num_layers= 6
트랜스포머에서 하나의 인코더와 디코더를 층으로 생각하였을 때, 트랜스포머 모델에서 인코더와 디코더가 총 몇 층으로 구성되었는지를 의미합니다. 논문에서는 인코더와 디코더를 각각 총 6개 쌓았습니다.

 num_heads= 8
트랜스포머에서는 어텐션을 사용할 때, 1번 하는 것 보다 여러 개로 분할해서 병렬로 어텐션을 수행하고 결과값을 다시 하나로 합치는 방식을 택했습니다. 이때 이 병렬의 개수를 의미합니다.

 dff= 2048
트랜스포머 내부에는 피드 포워드 신경망이 존재합니다. 이때 은닉층의 크기를 의미합니다. 피드 포워드 신경망의 입력층과 출력층의 크기는 
입니다.
```

이전 seq2seq 구조에서는 인코더와 디코더에서 각각 하나의 RNN이 t개의 시점(time-step)을 가지는 구조였다면 이번에는 인코더와 디코더라는 단위가 N개로 구성되는 구조입니다. 트랜스포머를 제안한 논문에서는 인코더와 디코더의 개수를 각각 6개를 사용하였습니다.




## 트랜스포머의 입력: 포지셔널 인코딩(Positional Encoding)

- RNN이 자연어 처리에서 유용했던 이유는 단어의 위치에 따라 단어를 순차적으로 입력받아서 처리하는 RNN의 특성으로 인해 각 단어의 위치 정보(position information)를 가질 수 있다는 점

- 하지만 트랜스포머는 단어 입력을 순차적으로 받는 방식이 아니므로 단어의 위치 정보를 다른 방식으로 알려줄 필요가 있습니다. 트랜스포머는 단어의 위치 정보를 얻기 위해서 각 단어의 임베딩 벡터에 위치 정보들을 더하여 모델의 입력으로 사용하는데, 이를 포지셔널 인코딩(positional encoding)이라고 합니다.
- 결국 트랜스포머의 입력은 순서 정보가 고려된 임베딩 벡터라고 보면 되겠습니다.


## 어텐션(Attention)

트랜스포머에서 사용되는 세 가지의 어텐션
- 인코더의 셀프 어텐션 : Query = Key = Value
- 디코더의 마스크드 셀프 어텐션 : Query = Key = Value
- 디코더의 인코더-디코더 어텐션 : Query : 디코더 벡터 / Key = Value : 인코더 벡터

## 인코더(Encoder)
- 트랜스포머는 하이퍼파라미터인  개수의 인코더 층을 쌓습니다. 논문에서는 총 6개의 인코더 층을 사용하였습니다.
- 하나의 인코더 층은 크게 총 2개의 서브층(sublayer)으로 나뉘어집니다. 바로 셀프 어텐션과 피드 포워드 신경망입니다. 
- 멀티 헤드 셀프 어텐션은 셀프 어텐션을 병렬적으로 사용하였다는 의미고, 포지션 와이즈 피드 포워드 신경망은 우리가 알고있는 일반적인 피드 포워드 신경망


### 인코더의 셀프 어텐션
-  셀프 어텐션이 앞서 배웠던 어텐션과 무엇이 다른지 이해해보겠습니다.

- 어텐션 함수는 주어진 '쿼리(Query)'에 대해서 모든 '키(Key)'와의 유사도를 각각 구합니다. 그리고 구해낸 이 유사도를 가중치로 하여 키와 맵핑되어있는 각각의 '값(Value)'에 반영해줍니다. 그리고 유사도가 반영된 '값(Value)'을 모두 가중합하여 리턴합니다.

- 여기까지는 앞서 배운 어텐션의 개념입니다. 그런데 어텐션 중에서는 셀프 어텐션(self-attention)이라는 것이 있습니다. 단지 어텐션을 자기 자신에게 수행한다는 의미입니다. 앞서 배운 seq2seq에서 어텐션을 사용할 경우의 Q, K, V의 정의를 다시 생각해봅시다.

```
Q = Query : t 시점의 디코더 셀에서의 은닉 상태
K = Keys : 모든 시점의 인코더 셀의 은닉 상태들
V = Values : 모든 시점의 인코더 셀의 은닉 상태들
```

그런데 사실 t 시점이라는 것은 계속 변화하면서 반복적으로 쿼리를 수행하므로 결국 전체 시점에 대해서 일반화를 할 수도 있습니다.

```
Q = Querys : 모든 시점의 디코더 셀에서의 은닉 상태들
K = Keys : 모든 시점의 인코더 셀의 은닉 상태들
V = Values : 모든 시점의 인코더 셀의 은닉 상태들
```

이처럼 기존에는 디코더 셀의 은닉 상태가 Q이고 인코더 셀의 은닉 상태가 K라는 점에서 Q와 K가 서로 다른 값을 가지고 있었습니다. 그런데 셀프 어텐션에서는 Q, K, V가 전부 동일합니다. 트랜스포머의 셀프 어텐션에서의 Q, K, V는 아래와 같습니다.

```
Q : 입력 문장의 모든 단어 벡터들
K : 입력 문장의 모든 단어 벡터들
V : 입력 문장의 모든 단어 벡터들
```


이후생략
https://wikidocs.net/31379

ㅋㅋㅋ 이게머야 ㅋㅋㅋ
wow Wow WWOWWWWWWW LOOLOOLOLOLOLOL

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf